In [41]:
# Borrowed from https://github.com/leeyeehoo/CSRNet-pytorch
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
from matplotlib import cm as CM
from tqdm import tqdm
from numba import cuda
# from image import *
%matplotlib inline

In [42]:
#this is borrowed from https://github.com/davideverona/deep-crowd-counting_crowdnet
# @cuda.autojit()
def gaussian_filter_density(gt):
#     print (gt.shape)
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

    pts = np.array(np.c_[np.nonzero(gt)[1], np.nonzero(gt)[0]])
    
    leafsize = 2048
    # build kdtree
    tree = scipy.spatial.KDTree(pts.copy(), leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k=4)

#     print ('generate density...')
    for i, pt in (enumerate(pts)):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1],pt[0]] = 1.
        if gt_count > 1:
            sigma = (distances[i][1]+distances[i][2]+distances[i][3])*0.1
        else:
            sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point
        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
#     print ('done.')
    return density

In [43]:
#set the root to the Shanghai dataset you download
root = '/home/rishhanth/Documents/gen_codes/CSRNet-tf/ShanghaiTech/'

In [44]:
#now generate the ShanghaiA's ground truth
part_A_train = os.path.join(root,'part_A/train_data','images')
part_A_test = os.path.join(root,'part_A/test_data','images')
part_B_train = os.path.join(root,'part_B/train_data','images')
part_B_test = os.path.join(root,'part_B/test_data','images')
path_sets = [part_A_train,part_A_test]

In [45]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [47]:
for img_path in tqdm(img_paths):
#     print (img_path)
    mat = io.loadmat(img_path.replace('.jpg','.mat').replace('images','ground-truth').replace('IMG_','GT_IMG_'))
    lab_path = img_path.replace('.jpg','.npy').replace('images','labels').replace('IMG_','LAB_')
    img= plt.imread(img_path)
    k = np.zeros((img.shape[0],img.shape[1]))
    gt = mat["image_info"][0,0][0,0][0]
    if not os.path.exists(path=lab_path):
#         continue
        for i in range(0,len(gt)):
            if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
                k[int(gt[i][1]),int(gt[i][0])]=1
    
        k = gaussian_filter_density(k)
        np.save(lab_path,k)
#     plt.imshow(k, cmap='jet')
#     plt.show()
#     with h5py.File(img_path.replace('.jpg','.h5').replace('images','ground_truth'), 'w') as hf:
#             hf['density'] = k



  0%|          | 0/482 [00:00<?, ?it/s]

  2%|▏         | 12/482 [00:00<00:03, 118.19it/s]

  5%|▍         | 24/482 [00:00<00:03, 117.72it/s]

  7%|▋         | 36/482 [00:00<00:03, 116.27it/s]

 10%|█         | 49/482 [00:00<00:03, 117.77it/s]

 13%|█▎        | 61/482 [00:00<00:03, 116.98it/s]

 16%|█▌        | 76/482 [00:00<00:03, 122.15it/s]

 16%|█▌        | 76/482 [00:18<01:37,  4.18it/s] 

 17%|█▋        | 84/482 [01:17<06:07,  1.08it/s]

 18%|█▊        | 85/482 [01:29<06:57,  1.05s/it]

 18%|█▊        | 86/482 [01:32<07:06,  1.08s/it]

 18%|█▊        | 87/482 [01:36<07:15,  1.10s/it]

 18%|█▊        | 88/482 [01:43<07:45,  1.18s/it]

 18%|█▊        | 89/482 [01:51<08:14,  1.26s/it]

 19%|█▊        | 90/482 [01:54<08:17,  1.27s/it]

 19%|█▉        | 91/482 [02:18<09:54,  1.52s/it]

 19%|█▉        | 92/482 [02:18<09:48,  1.51s/it]

 19%|█▉        | 93/482 [02:37<10:57,  1.69s/it]

 20%|█▉        | 94/482 [02:38<10:54,  1.69s/it]

 20%|█▉        | 95/482 [02:47<11:21,  1.76s/it]



 46%|████▌     | 220/482 [32:12<38:21,  8.78s/it]

 46%|████▌     | 221/482 [32:26<38:18,  8.81s/it]

 46%|████▌     | 222/482 [32:29<38:02,  8.78s/it]

 46%|████▋     | 223/482 [32:47<38:05,  8.82s/it]

 46%|████▋     | 224/482 [32:57<37:58,  8.83s/it]

 47%|████▋     | 225/482 [33:05<37:47,  8.82s/it]

 47%|████▋     | 226/482 [33:31<37:58,  8.90s/it]

 47%|████▋     | 227/482 [33:50<38:01,  8.95s/it]

 47%|████▋     | 228/482 [34:14<38:08,  9.01s/it]

 48%|████▊     | 229/482 [34:21<37:57,  9.00s/it]

 48%|████▊     | 230/482 [34:22<37:40,  8.97s/it]

 48%|████▊     | 231/482 [34:30<37:29,  8.96s/it]

 48%|████▊     | 232/482 [34:37<37:18,  8.95s/it]

 48%|████▊     | 233/482 [34:38<37:01,  8.92s/it]

 49%|████▊     | 234/482 [34:43<36:48,  8.91s/it]

 49%|████▉     | 235/482 [35:05<36:53,  8.96s/it]

 49%|████▉     | 236/482 [35:09<36:39,  8.94s/it]

 49%|████▉     | 237/482 [35:30<36:42,  8.99s/it]

 49%|████▉     | 238/482 [35:31<36:24,  8.95s/it]

 50%|████▉     | 239/482 [35:43

KeyboardInterrupt: 

In [ ]:
#now see a sample from ShanghaiA
plt.imshow(Image.open(img_paths[0]))

In [ ]:
gt_file = h5py.File(img_paths[0].replace('.jpg','.h5').replace('images','ground_truth'),'r')
groundtruth = np.asarray(gt_file['density'])
plt.imshow(groundtruth,cmap=CM.jet)

In [ ]:
np.sum(groundtruth)# don't mind this slight variation

In [ ]:
#now generate the ShanghaiB's ground truth
path_sets = [part_B_train,part_B_test]

In [ ]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [ ]:
for img_path in img_paths:
    print img_path
    mat = io.loadmat(img_path.replace('.jpg','.mat').replace('images','ground_truth').replace('IMG_','GT_IMG_'))
    img= plt.imread(img_path)
    k = np.zeros((img.shape[0],img.shape[1]))
    gt = mat["image_info"][0,0][0,0][0]
    for i in range(0,len(gt)):
        if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
            k[int(gt[i][1]),int(gt[i][0])]=1
    k = gaussian_filter(k,15)
    with h5py.File(img_path.replace('.jpg','.h5').replace('images','ground_truth'), 'w') as hf:
            hf['density'] = k